#Парсинг статей Habr

План:

1.Импорт

2.Получить список url

3.Функция парсинга и аполнения списков

4.формирование csv

5.Xtrain Ytrain


## Import

In [0]:
# Импорт библиотек для парсинга
import requests # получаем данные по ссылке
from bs4 import BeautifulSoup # парсим документ
import time # пауза в загрузке данных
import pandas as pd

## Получаем список URL

In [0]:
#создаем список URL
url = 'https://habr.com/ru' # базовая ссылка
firstPage = 352504 # номер  страницы с первым постом
lastPage = 353139 # номер страницы с последним постом

# создаеим фукнцию перебора страниц  и записи адресов в список
def get_urls():
  listUrl = [] # создаем пустой список
  for page in range(firstPage, lastPage):
    pageUrl = '{}/post/{}'.format(url, page)
    listUrl.append(pageUrl)
  return listUrl # возвращаем заполненный список

In [0]:
# смотрим количество ссылок
len(get_url20())

635

## Функция парсинга с сайта 

In [0]:
# создаем функцию парсинга всех статей

titlesList = [] # создаем список заголовков
categoriesList = [] # создаем список категорий
postsList = [] # создаем список текста статей
karmasList = [] # список кармы
followList = [] # общий список для подписчиков и подписок, т.к. они в одном контейнере
followersList = [] # список подписчиков
subscribesList = [] # список подписок автора
ratingsList = [] # список рейтингов



def get_data():
   
  for pageData in get_urls():
    fullData = requests.get(pageData) # отправляем запрос
    
    if fullData.status_code == 200: # если ссылка рабочая, код = 200
      
      soup = BeautifulSoup(fullData.text, 'html5lib') # парсим страницу поста
      
      # Заполняем списки заголовок
      titles = soup.findAll('h1', class_='post__title post__title_full') # парсим заголовок
      
      for title in range(len(titles)): 
        if titles[title].find('span', class_ = 'post__title-text') is not None: # если заголовок не пустой
          titlesList.append(titles[title].text.replace('!',' ').replace('?','').replace('-','').replace('.','').replace("\n      ", "").replace("\n    ", "").lower()) # добавляем  в список, очищаем строки
      
      # Заполняем списки категорий    
      categories = soup.findAll('ul', class_ = 'post__hubs post__hubs_full-post inline-list') # парсим категории
      
      for category in range(len(categories)):
        if categories[category].find('li', class_ = 'inline-list__item inline-list__item_hub') is not None: # если категория не пустая
          categoriesList.append(categories[category].text.replace(',','').replace('!',' ').replace('?','').replace('-','').replace('.','').replace('\n          \n            ', "").replace('\n          ',"").replace("\n      ","").lower()) # добавляем в список, очищаем
      
      # Заполняем списки постов
      posts = soup.findAll('div', class_ = 'post__body post__body_full')
      
      for post in range(len(posts)):
        if posts[post].find('div', class_ = 'post__text post__text-html js-mediator-article') is not None:
          postsList.append(posts[post].text.replace('!',' ').replace('?','').replace('-','').replace('.','').replace("\n      ", "").replace("\n\n", "").replace("\n", " ").lower())
      
      # список кармы
      karmas = soup.findAll('a', class_ = 'user-info__stats-item stacked-counter')
      
      for karma in range(len(karmas)):
        if karmas[karma].find('div', class_ = 'stacked-counter__value stacked-counter__value_green ') is not None:
          karmasList.append(karmas[karma].text.replace('\n            ','').replace('\n            Карма\n          ','').replace('Карма\n          ',''))
      
      # список рейтинга статьи
      ratings = soup.findAll('a', class_ = 'user-info__stats-item stacked-counter stacked-counter_rating')
      
      for rating in range(len(ratings)):
        if ratings[rating].find('div', class_ = 'stacked-counter__value stacked-counter__value_magenta') is not None:
          ratingsList.append(ratings[rating].text.replace('\n','').replace('Рейтинг','').replace(' ','').replace('1\xa00',''))
          
      # список подписчиков и подписок автора
      followers = soup.findAll('a', class_ ='user-info__stats-item stacked-counter stacked-counter_subscribers')
      
      for follower in range(len(followers)):
        if followers[follower].find('div',class_ = 'stacked-counter__value stacked-counter__value_blue') is not None:
          followList.append(followers[follower].text.replace(' ','').replace('\n', '').replace('Подписчики','').replace('Подписки', ''))
      
    time.sleep(2.0) # пауза для обхода блокировки 
    
    
  return titlesList, categoriesList, postsList, karmasList, ratingsList, followList
get_data()
# функция возвращает статьи с Habr, очищенные и приведенные в нижний регистр

Output hidden; open in https://colab.research.google.com to view.

In [0]:
subscribesList = followList[1::2] # Подписки - добавляем из списка followList, т.к. они лежат в одном контейнере
followersList = followList[0::2] # Подписчики - добавляем из списка followList

In [0]:
# создаем словарь для датафрейма
data_dict = {'title': titlesList, 'categorie' : categoriesList, 'post' : postsList, 'karma' : karmasList, 'rating' : ratingsList, 'followers' : followersList, 'subscribes' : subscribesList}

## Создание датфрейма и csv

In [0]:
dataHabr = pd.DataFrame.from_dict(data_dict, orient='index') # создаем из словаря data_dict
dataHabr = dataHabr.transpose() # поворачиваем, чтобы индексы были сверху
dataHabr.head() # заголовок

,title,categorie,post,karma,rating,followers,subscribes
0,рубрика «читаем статьи за вас» декабрь 2017 — ...,блог компании open data science алгоритмы обра...,"привет, хабр продолжаем публиковать рецензии...","99,5","2,6",220,16
1,тренды безопасности: почему злоумышленники ата...,блог компании iti capital информационная безоп...,хакеры становятся всё более изобретательными в...,"10,0","186,0",375,0
2,как мы в smart engines учили sailfish os распо...,блог компании smart engines программирование р...,всем привет как вы уже знаете по нашим стать...,"48,0","6,4",49,1
3,что новенького в swift 41,разработка под ios разработка мобильных прилож...,этот пост является вольным переводом статьи w...,"11,2","0,0",69,9
4,альтернативные подходы к разработке новых ит п...,анализ и проектирование систем разработка моби...,"это продолжение статьи: что делать, если нужно...","15,0","35,2",8,1


In [0]:
dataHabr.shape

(275, 7)

In [0]:
dataHabr.to_csv("habr02.csv", sep=',', index = False) # сохраняем csv

In [0]:
# Качаем csv на диск
from google.colab import files
files.download('habr02.csv')